In [20]:
! pip install frictionless

In [35]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import time
from getpass import getpass
from ftplib import FTP_TLS
from ftplib import FTP
import ftplib
import frictionless
import ast

In [22]:
import pandas as pd
import re
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

def extract_id(fileName):
    ogd_id = fileName.partition('_')[0]
    return ogd_id.replace('ogd', '')

def getInformation(url):

  csvList = []
  url = url.replace(" ","%20")
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))

  for elem in x:
    stringElem = str(elem)

    if 'csv' in stringElem:
      fileName = re.search('href="(.*)"', stringElem).group(1)
      csvList.append([fileName, extract_id(fileName)])

  df = pd.DataFrame(csvList, columns= ['fileName', 'ogd_id'])
  print(df)
  return df
  #df.to_csv('files/info.csv')

df = getInformation(url)

                                            fileName ogd_id
0  ogd35_schweizerische_elektrizitaetsbilanz_mona...     35
1                                     ogd39_epec.csv     39
2                     ogd47_vollzugsresultate_pw.csv     47
3                ogd47_vollzugsresultate_pw_2020.csv     47
4                    ogd61_vollzugsresultate_lnf.csv     61
5               ogd61_vollzugsresultate_lnf_2020.csv     61
6           ogd90_potenzialstudien_waermenutzung.csv     90


-----------------------------------------------------------------------------------------------

In [23]:
csv = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv'

In [24]:
import pandas as pd
data = pd.read_csv(csv)

In [25]:
data

,Jahr,Monat,Definitiv,Erzeugung_laufwerk_GWh,Erzeugung_speicherwerk_GWh,Erzeugung_kernkraftwerk_GWh,Erzeugung_andere_GWh,Verbrauch_speicherpumpen_GWh,Erzeugung_netto_GWh,Einfuhr_GWh,Ausfuhr_GWh,Landesverbrauch_GWh,Verluste_GWh,Endverbrauch_GWh
0,2000,1,1,891,2004,2354,238,72,5415,2580,2521,5474,346,5128
1,2000,2,1,959,1516,2213,232,40,4880,2478,2337,5021,353,4668
2,2000,3,1,1069,1463,2355,229,44,5072,2608,2596,5084,347,4737
3,2000,4,1,1196,1068,2278,218,139,4621,2181,2375,4427,334,4093
4,2000,5,1,2060,1920,2307,197,228,6256,1212,3034,4434,315,4119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2022,12,1,944,1706,2200,437,593,4694,3283,2140,5837,392,5445
276,2023,1,1,1003,2075,2202,439,385,5334,3396,2826,5904,371,5533
277,2023,2,1,673,1748,1983,508,303,4609,3320,2674,5255,371,4884
278,2023,3,0,986,1749,2191,592,355,5163,2685,2420,5428,376,5052


In [26]:
from frictionless import Package, validate

In [27]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

    folderPath = url_ogd + row['ogd_id'] + '/'
    datapackagePath = folderPath + dataPackage

    print(row['ogd_id'])

    #print(datapackagePath)

    # store the response of URL
    response = urlopen(datapackagePath)

    # storing the JSON response
    # from url in data


    data_json = json.loads(response.read())


    #data_json['resources'][0]['path'] = url_ogd + staging + row['fileName']


    # print the json response
    print(data_json)

    # save the file as data.json
    with open('data.json', 'w') as f:
      json.dump(data_json, f)

    datapackage = Package('data.json')

    report = validate('data.json', scheme='C:\\_TMP\\Test.json')

    # test the data
    report = frictionless.validate(datapackage)

    print(report.tasks[0].errors)

    if not(report.valid):
      #send email with report message
      #remove file from staging folder
      print("Not valid!")

    else:
      print("Valid.")

    print()

35
{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/35/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': 'Année', 'description@de': 'Jahr der Aufnahme', 'description@fr': "Année d'acquisition"}, {'name': 'Monat', 'type': 'integer', 'format': 'default', 'title@de': 'Monat', 'title@fr': 'Mois', 'description@de': 'Monat der Aufnahme', 'description@fr': "Mois d'acquisition", 'constraints': {'minimum': '1', 'maximum': '12'}}, {'name': 'Definitiv', 'type': 'boolean', 'format': 'default', 'title@de': '', 'title@fr': '', 'description@de': '', 'description@fr': '', 'trueValues': ['1'], 'falseValues': ['0']}, {'name': 'Erzeugung_laufwerk_GWh', 'type': 'integer', 'format': 'default', 'title@de': 'Erzeugung Laufwerk GWh', 'title@f

---------------------------------------------

Trying it without saving  the json in a separate file.

In [28]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

    folderPath = url_ogd + row['ogd_id'] + '/'
    datapackagePath = folderPath + dataPackage

    #print(datapackagePath)

    # store the response of URL
    response = urlopen(datapackagePath)

    # storing the JSON response
    # from url in data
    data_json = json.loads(response.read())
    data_json['resources'][0]['path'] = url_ogd + staging + row['fileName']

    # print the json response
    print(data_json)

    # test the data
    report = frictionless.validate(data_json)

    datapackage = Package('data.json')

    if not(report.valid):
      #send email with report message
      #remove file from staging folder
      print("Not valid!")

      for error in report.flatten():
        print('Error: ', error.get('code'))
        print('Message: ', error.get('message'))
        print('Row: ', error.get('row'))
        print('Col: ', error.get('column'))
        print('Cell Value: '. error.get('cell'))
    else:
      print("Valid.")

      #move the file to the correct ogd folder
      # maybe overwrite or remove the old file

    print()


{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': 'Année', 'description@de': 'Jahr der Aufnahme', 'description@fr': "Année d'acquisition"}, {'name': 'Monat', 'type': 'integer', 'format': 'default', 'title@de': 'Monat', 'title@fr': 'Mois', 'description@de': 'Monat der Aufnahme', 'description@fr': "Mois d'acquisition", 'constraints': {'minimum': '1', 'maximum': '12'}}, {'name': 'Definitiv', 'type': 'boolean', 'format': 'default', 'title@de': '', 'title@fr': '', 'description@de': '', 'description@fr': '', 'trueValues': ['1'], 'falseValues': ['0']}, {'name': 'Erzeugung_laufwerk_GWh', 'type': 'integer', 'format': 'default', 'title@de': 'Erzeugung Laufwerk GWh', 'title

In [29]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

    folderPath = url_ogd + row['ogd_id'] + '/'
    datapackagePath = folderPath + dataPackage

    print(row['ogd_id'])

    #print(datapackagePath)

    # store the response of URL
    response = urlopen(datapackagePath)

    # storing the JSON response
    # from url in data

    data_json = json.loads(response.read())

    # change source file
    #data_json['resources'][0]['path'] = url_ogd + staging + row['fileName']


    # print the json response
    print(data_json)

    # save the file as data.json
    with open('data.json', 'w') as f:
      json.dump(data_json, f)

    report = validate(url_ogd + staging + row['fileName'], scheme='data.json')

    #print(report.tasks[0].errors)

    if not(report.valid):
      print("Not valid!")
    else:
      print("Valid.")

    print()

35
{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/35/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': 'Année', 'description@de': 'Jahr der Aufnahme', 'description@fr': "Année d'acquisition"}, {'name': 'Monat', 'type': 'integer', 'format': 'default', 'title@de': 'Monat', 'title@fr': 'Mois', 'description@de': 'Monat der Aufnahme', 'description@fr': "Mois d'acquisition", 'constraints': {'minimum': '1', 'maximum': '12'}}, {'name': 'Definitiv', 'type': 'boolean', 'format': 'default', 'title@de': '', 'title@fr': '', 'description@de': '', 'description@fr': '', 'trueValues': ['1'], 'falseValues': ['0']}, {'name': 'Erzeugung_laufwerk_GWh', 'type': 'integer', 'format': 'default', 'title@de': 'Erzeugung Laufwerk GWh', 'title@f

----------------------------------------------------------------

------------------------------------------------------------------------------------------

In [31]:
import os
import shutil
from frictionless import validate

Test the original files:

In [32]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
url = 'https://www.uvek-gis.admin.ch/BFE/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

  # load original data package
  folderPath = url_ogd + row['ogd_id'] + '/'
  datapackagePath = os.path.join(folderPath, dataPackage)

  report = validate(datapackagePath)

  print(row['ogd_id'])
  print(report.valid, '\n')

35
True 

39
True 

47
True 

47
True 

61
True 

61
True 

90
True 



Test the adjusted files:

In [38]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
url = 'https://www.uvek-gis.admin.ch/BFE/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

  # load original data package
  folderPath = url_ogd + row['ogd_id'] + '/'
  datapackagePath = os.path.join(folderPath, dataPackage)

  # new file path --> to staging folder
  stagingFilePath = url_ogd + 'staging/' + row['fileName']

  # store the response of URL
  response = urlopen(datapackagePath)

  # storing the JSON response
  data_json = json.loads(response.read())



  # change source file
  jsonAsString = str(data_json)
  jsonAsString = jsonAsString.replace(os.path.join(folderPath, row['fileName']), stagingFilePath)
  updatedJSON = ast.literal_eval(jsonAsString)

  # path for temporary datapackage
  tmpJSON_path = 'tmpPackage.json'

  # save the file as data.json
  with open(tmpJSON_path, 'w') as f:
    json.dump(updatedJSON, f)

  report = validate(tmpJSON_path)

  if report.valid:
    print("Valid.")

    # move file to corresponding ogd folder
    shutil.move(stagingFilePath, os.path.join(folderPath, row['fileName']))

  else:
    print("Not valid!")

    error_details = []

    for error in report.flatten(['rowPosition', 'fieldPosition', 'message']):
      error_details.append(f"Row: {error['rowPosition']}, Column: {error['fieldPosition']}, Message: {error['message']}")

    email_body = 'The data file ' + row['fileName'] + ' is not valid. Please check the date.\n\n'
    email_body += '\n'.join(error_details)

    subject = "Data Validation Error - " + row['ogd_id']

Valid.


FileNotFoundError: ignored